In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
import pynvml

# Define paths
CSV_FILE = "R_training_2025-03-26.csv"

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
batch_size = 64
epochs = 300
lr = 0.0001
latent_dim = 100
image_size = 256

# Initialize GPU monitoring
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

def gpu_usage():
    info = pynvml.nvmlDeviceGetUtilizationRates(handle)
    memory = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU Usage: {info.gpu}% | Memory Usage: {memory.used / 1024**2:.2f} MB")

# Custom dataset loader
class RetinalDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file)
        self.image_paths = df["preprocessed_image"].tolist()
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = np.load(img_path)
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # Convert to tensor, rearrange channels
        return img.to(device), 0  # Dummy label

# Load dataset using the CSV file
dataset_right = RetinalDataset(CSV_FILE)
dataloader_right = DataLoader(dataset_right, batch_size=batch_size, shuffle=True)

# Define Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, image_size * image_size * 3),
            nn.Tanh()
        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(-1, 3, image_size, image_size)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size * image_size * 3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Training function
def train_gan(dataloader, eye):
    print(f"\n🔵 Training GAN for {eye} eye images on {device}...")
    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)
    adversarial_loss = nn.BCELoss().to(device)
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    start_time = time.time()
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        for i, (imgs, _) in enumerate(dataloader):
            batch_size = imgs.size(0)
            real_imgs = imgs.to(device)
            
            # Adversarial ground truths
            valid = torch.ones(batch_size, 1, device=device)
            fake = torch.zeros(batch_size, 1, device=device)
            
            # Train Generator
            optimizer_G.zero_grad()
            z = torch.randn(batch_size, latent_dim, device=device)
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()
            
            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
        
        epoch_time = time.time() - epoch_start
        gpu_usage()
        print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | Time: {epoch_time:.2f} sec")
    
    total_time = time.time() - start_time
    print(f"✅ {eye.capitalize()} Eye GAN Training Complete! Total Training Time: {total_time:.2f} sec")
    
    today = datetime.today().strftime('%Y-%m-%d')
    torch.save(generator.state_dict(), f"generator_right_{eye}_{today}.pth")
    torch.save(discriminator.state_dict(), f"discriminator_right_{eye}_{today}.pth")
    
    print(f"📝 Model saved: generator_{eye}_{today}.pth and discriminator_{eye}_{today}.pth")

# Train GAN for right eye using the CSV file
train_gan(dataloader_right, "right")

print("🎯 Training Finished! Model saved for right eye.")

Using device: cuda

🔵 Training GAN for right eye images on cuda...
GPU Usage: 71% | Memory Usage: 8893.96 MB
Epoch 1/300 | D Loss: 3.4539 | G Loss: 0.0012 | Time: 3.95 sec
GPU Usage: 70% | Memory Usage: 8893.96 MB
Epoch 2/300 | D Loss: 0.4314 | G Loss: 0.5496 | Time: 3.70 sec
GPU Usage: 71% | Memory Usage: 8893.96 MB
Epoch 3/300 | D Loss: 0.4743 | G Loss: 0.5231 | Time: 3.63 sec
GPU Usage: 69% | Memory Usage: 8893.96 MB
Epoch 4/300 | D Loss: 0.3308 | G Loss: 0.8617 | Time: 3.74 sec
GPU Usage: 66% | Memory Usage: 8893.96 MB
Epoch 5/300 | D Loss: 0.7000 | G Loss: 0.2871 | Time: 3.68 sec
GPU Usage: 61% | Memory Usage: 8893.96 MB
Epoch 6/300 | D Loss: 0.6710 | G Loss: 0.3147 | Time: 4.85 sec
GPU Usage: 63% | Memory Usage: 8893.96 MB
Epoch 7/300 | D Loss: 0.5338 | G Loss: 0.5983 | Time: 3.69 sec
GPU Usage: 68% | Memory Usage: 8893.96 MB
Epoch 8/300 | D Loss: 0.5440 | G Loss: 0.6891 | Time: 3.63 sec
GPU Usage: 74% | Memory Usage: 8893.96 MB
Epoch 9/300 | D Loss: 0.6503 | G Loss: 0.3884 | Tim

# Testing

In [5]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Define paths
CSV_FILE = "R_testing_2025-03-26.csv"
GENERATOR_PATH = "generator_right_right_2025-03-26.pth"  # Update with correct path
LATENT_DIM = 100
IMAGE_SIZE = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load trained Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, IMAGE_SIZE * IMAGE_SIZE * 3),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(-1, 3, IMAGE_SIZE, IMAGE_SIZE)
        return img

# Initialize and load the generator
generator = Generator(LATENT_DIM).to(DEVICE)
generator.load_state_dict(torch.load(GENERATOR_PATH, map_location=DEVICE, weights_only=True))
generator.eval()

# Load test dataset
df = pd.read_csv(CSV_FILE)

# Ensure output directory exists
os.makedirs("generated_right", exist_ok=True)

# Run anomaly detection on test images
results = []
for _, row in df.iterrows():
    img_path = row["preprocessed_image"]
    dr_grade = row["dr_grades"]
    image_id = row["image_id"]
    
    # Load test image (Expected: [0,1] range)
    test_img = torch.tensor(np.load(img_path), dtype=torch.float32).permute(2, 0, 1).to(DEVICE)

    # Normalize from [0,1] -> [-1,1] (GAN expects this input range)
    test_img = (test_img * 2) - 1

    # Generate synthetic image
    z = torch.randn(1, LATENT_DIM, device=DEVICE)
    gen_img = generator(z).detach().cpu().numpy().squeeze(0)

    # Convert shape from (3, 256, 256) -> (256, 256, 3) for saving
    gen_img = np.transpose(gen_img, (1, 2, 0))

    # Fix shape of test_img for L1 loss calculation
    test_img_np = test_img.cpu().numpy().transpose(1, 2, 0)  # Convert to (256, 256, 3)

    # Compute Anomaly Score (L1 Loss)
    anomaly_score = np.mean(np.abs(test_img_np - gen_img))

    # Save generated image
    output_path = f"generated_right/{image_id}_generated.png"
    plt.imsave(output_path, np.clip((gen_img + 1) / 2, 0, 1))  # Convert back to [0,1] for saving

    # Store results
    results.append([image_id, dr_grade, anomaly_score])

    print(f"Processed {image_id} | DR Grade: {dr_grade} | Anomaly Score: {anomaly_score:.4f}")

# Save results to CSV
result_df = pd.DataFrame(results, columns=["image_id", "dr_grades", "anomaly_score"])
result_df.to_csv("anomaly_results_right.csv", index=False)

print("\n✅ Testing Complete! Results saved to anomaly_results_right.csv")


Processed 2_1_R.jpg | DR Grade: 0 | Anomaly Score: 0.6980
Processed 19_9_R.jpg | DR Grade: 0 | Anomaly Score: 0.6774
Processed 28_1_R.jpg | DR Grade: 0 | Anomaly Score: 0.7888
Processed 51_3_R.jpg | DR Grade: 0 | Anomaly Score: 0.6619
Processed 79_4_R.jpg | DR Grade: 0 | Anomaly Score: 0.7767
Processed 82_2_R.jpg | DR Grade: 0 | Anomaly Score: 0.7343
Processed 83_6_R.jpg | DR Grade: 0 | Anomaly Score: 0.7140
Processed 106_3_R.jpg | DR Grade: 0 | Anomaly Score: 0.8755
Processed 109_3_R.jpg | DR Grade: 0 | Anomaly Score: 0.7447
Processed 121_1_R.jpg | DR Grade: 0 | Anomaly Score: 0.6888
Processed 133_1_R.jpg | DR Grade: 0 | Anomaly Score: 0.8297
Processed 134_1_R.jpg | DR Grade: 0 | Anomaly Score: 0.6759
Processed 159_4_R.jpg | DR Grade: 0 | Anomaly Score: 0.5731
Processed 160_5_R.jpg | DR Grade: 0 | Anomaly Score: 0.7707
Processed 163_3_R.jpg | DR Grade: 0 | Anomaly Score: 0.6098
Processed 168_1_R.jpg | DR Grade: 0 | Anomaly Score: 0.6770
Processed 169_1_R.jpg | DR Grade: 0 | Anomaly Sc

# using sigmord of generator

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
import pynvml

# Define paths
CSV_FILE = "R_training_2025-03-26.csv"

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define hyperparameters
batch_size = 64
epochs = 1000
lr = 0.0001
latent_dim = 100
image_size = 256

# Initialize GPU monitoring
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

def gpu_usage():
    info = pynvml.nvmlDeviceGetUtilizationRates(handle)
    memory = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU Usage: {info.gpu}% | Memory Usage: {memory.used / 1024**2:.2f} MB")

# Custom dataset loader
class RetinalDataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file)
        self.image_paths = df["preprocessed_image"].tolist()
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = np.load(img_path)
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)  # Convert to tensor, rearrange channels
        return img.to(device), 0  # Dummy label

# Load dataset using the CSV file
dataset_right = RetinalDataset(CSV_FILE)
dataloader_right = DataLoader(dataset_right, batch_size=batch_size, shuffle=True)

# Define Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, image_size * image_size * 3),
            nn.Sigmoid()

        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(-1, 3, image_size, image_size)
        return img

# Define Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size * image_size * 3, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

# Training function
def train_gan(dataloader, eye):
    print(f"\n🔵 Training GAN for {eye} eye images on {device}...")
    generator = Generator(latent_dim).to(device)
    discriminator = Discriminator().to(device)
    adversarial_loss = nn.BCELoss().to(device)
    optimizer_G = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    start_time = time.time()
    
    for epoch in range(epochs):
        epoch_start = time.time()
        
        for i, (imgs, _) in enumerate(dataloader):
            batch_size = imgs.size(0)
            real_imgs = imgs.to(device)
            
            # Adversarial ground truths
            valid = torch.ones(batch_size, 1, device=device)
            fake = torch.zeros(batch_size, 1, device=device)
            
            # Train Generator
            optimizer_G.zero_grad()
            z = torch.randn(batch_size, latent_dim, device=device)
            gen_imgs = generator(z)
            g_loss = adversarial_loss(discriminator(gen_imgs), valid)
            g_loss.backward()
            optimizer_G.step()
            
            # Train Discriminator
            optimizer_D.zero_grad()
            real_loss = adversarial_loss(discriminator(real_imgs), valid)
            fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2
            d_loss.backward()
            optimizer_D.step()
        
        epoch_time = time.time() - epoch_start
        gpu_usage()
        print(f"Epoch {epoch+1}/{epochs} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | Time: {epoch_time:.2f} sec")
    
    total_time = time.time() - start_time
    print(f"✅ {eye.capitalize()} Eye GAN Training Complete! Total Training Time: {total_time:.2f} sec")
    
    today = datetime.today().strftime('%Y-%m-%d')
    torch.save(generator.state_dict(), f"sigmoid_1000_generator_right_{eye}_{today}.pth")
    torch.save(discriminator.state_dict(), f"sigmoid_1000_discriminator_right_{eye}_{today}.pth")
    
    print(f"📝 Model saved: sigmoid_1000_generator_{eye}_{today}.pth and sigmoid_1000_discriminator_{eye}_{today}.pth")

# Train GAN for right eye using the CSV file
train_gan(dataloader_right, "right")

print("🎯 Training Finished! Model saved for right eye.")

Using device: cuda

🔵 Training GAN for right eye images on cuda...
GPU Usage: 88% | Memory Usage: 18767.86 MB
Epoch 1/1000 | D Loss: 1.5692 | G Loss: 6.2378 | Time: 7.22 sec
GPU Usage: 91% | Memory Usage: 18767.86 MB
Epoch 2/1000 | D Loss: 0.3958 | G Loss: 0.6750 | Time: 7.61 sec
GPU Usage: 93% | Memory Usage: 18767.86 MB
Epoch 3/1000 | D Loss: 0.6855 | G Loss: 1.0107 | Time: 7.66 sec
GPU Usage: 79% | Memory Usage: 18767.86 MB
Epoch 4/1000 | D Loss: 0.4710 | G Loss: 1.6253 | Time: 8.85 sec
GPU Usage: 95% | Memory Usage: 18767.86 MB
Epoch 5/1000 | D Loss: 0.5099 | G Loss: 1.1448 | Time: 7.02 sec
GPU Usage: 87% | Memory Usage: 18767.86 MB
Epoch 6/1000 | D Loss: 0.5322 | G Loss: 1.0767 | Time: 7.99 sec
GPU Usage: 82% | Memory Usage: 18767.86 MB
Epoch 7/1000 | D Loss: 0.5216 | G Loss: 1.1075 | Time: 7.44 sec
GPU Usage: 88% | Memory Usage: 18767.86 MB
Epoch 8/1000 | D Loss: 0.7807 | G Loss: 1.7504 | Time: 8.73 sec
GPU Usage: 85% | Memory Usage: 18767.86 MB
Epoch 9/1000 | D Loss: 0.5136 | G 

# Testing

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Define paths
CSV_FILE = "R_testing_2025-03-26.csv"  # Right eye dataset
GENERATOR_PATH = "sigmoid_1000_generator_right_right_2025-03-26.pth"  # Right eye generator model
LATENT_DIM = 100
IMAGE_SIZE = 256
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load trained Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, IMAGE_SIZE * IMAGE_SIZE * 3),
            nn.Sigmoid()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(-1, 3, IMAGE_SIZE, IMAGE_SIZE)
        return img

# Initialize and load the generator
generator = Generator(LATENT_DIM).to(DEVICE)
generator.load_state_dict(torch.load(GENERATOR_PATH, map_location=DEVICE))
generator.eval()

# Load test dataset
df = pd.read_csv(CSV_FILE)

# Ensure output directory exists
os.makedirs("generated_right_1000_sigmoid", exist_ok=True)

# Run anomaly detection on test images
results = []
for _, row in df.iterrows():
    img_path = row["preprocessed_image"]
    dr_grade = row["dr_grades"]
    image_id = row["image_id"]
    
    # Load test image (Expected: [0,1] range)
    test_img = torch.tensor(np.load(img_path), dtype=torch.float32).permute(2, 0, 1).to(DEVICE)

    # Generate synthetic image
    z = torch.randn(1, LATENT_DIM, device=DEVICE)
    gen_img = generator(z).detach().cpu().numpy().squeeze(0)

    # Convert shape from (3, 256, 256) -> (256, 256, 3) for saving
    gen_img = np.transpose(gen_img, (1, 2, 0))

    # Fix shape of test_img for L1 loss calculation
    test_img_np = test_img.cpu().numpy().transpose(1, 2, 0)  # Convert to (256, 256, 3)

    # Compute Anomaly Score (L1 Loss)
    anomaly_score = np.mean(np.abs(test_img_np - gen_img))

    # Save generated image
    output_path = f"generated_right_1000_sigmoid/{image_id}_generated.png"
    plt.imsave(output_path, np.clip(gen_img, 0, 1))  # No need to re-normalize

    # Store results
    results.append([image_id, dr_grade, anomaly_score])

    print(f"Processed {image_id} | DR Grade: {dr_grade} | Anomaly Score: {anomaly_score:.4f}")

# Save results to CSV
result_df = pd.DataFrame(results, columns=["image_id", "dr_grades", "anomaly_score"])
result_df.to_csv("anomaly_1000_sigmoid_results_right.csv", index=False)

print("\n✅ Testing Complete! Results saved to anomaly_1000_sigmoid_results_right.csv")
